In [ ]:
##
# Dataset preprocessing

from pandas import read_csv

# Read dataset
dataset_csv = read_csv(r'./datasets/obfuscated_dataset_v1.1.csv')

# Drop function name
dataset_csv = dataset_csv.drop(columns=['function name'])

# Fill target as 'not related'=0
dataset_csv[['target']] = dataset_csv[['target']].fillna(value=0)

In [ ]:

import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Define 2 version, normal and 'prod' which will be used
PROD = True

# Sepparate data into attributes and labels
features = dataset_csv.drop('target', axis=1)
target = dataset_csv['target']

pickle_name = ''

if (not PROD):
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.15)
    pickle_name = 'whitebox_svm_classifier.pkl'
else:
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20)
    pickle_name = 'whitebox_svm_classifier_prod.pkl'
    X_train = features
    y_train = target

# Add weights to features
weights = np.ones(X_train.shape[0])
weights[X_train.iloc[:, 7] == 1] = 2

# Train with tuned hyperparameters
classifier = SVC(kernel='linear', C=0.1)
classifier.fit(X_train, y_train, sample_weight = weights)

y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

# Save the classifier
with open(pickle_name, "wb") as f:
    pickle.dump(classifier, f)
